In [13]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D

model=Sequential()

model.add(Conv2D(256, (3, 3),input_shape=[50, 50, 1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(64, activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(4, activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [14]:
model.load_weights('../res/Models/modelAgeGender.h5')

import os, pickle
import cv2, dlib
import numpy as np

In [15]:
face_detector = dlib.get_frontal_face_detector()
margin = 20

In [16]:
test_path='../res/test_data/images'

img_names=os.listdir(test_path)
print(img_names)

dict_file = open("../res/Data/cat_dict.pkl", "rb")
category_dict = pickle.load(dict_file)

for img_name in img_names:
    img = cv2.imread(os.path.join(test_path,img_name))
    faces = face_detector(img)
    
    try:
        for face in faces:

            x1 = (face.left()) - margin
            y1 = (face.top()) - margin
            x2 = (face.right()) + margin
            y2 = (face.bottom()) + margin

            test_img = img[y1:y2, x1:x2]

            test_img = cv2.resize(test_img, (50,50))
            test_img = cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
            test_img = test_img/255
            test_img = test_img.reshape(1, 50, 50, 1)

            results = model.predict(test_img)
            label = np.argmax(results, axis=1)[0]
            acc = int(np.max(results, axis=1)[0]*100)

            print(results, label, category_dict[label],acc)

            cv2.rectangle(img, (x1, y1), (x2, y2),(0, 255, 0), 2)

            cv2.putText(img, f"{category_dict[label]}({str(acc)}%)", (x1, y1-1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    except Exception as e:
        print(e)
    
    cv2.imshow('LIVE',img)
    k=cv2.waitKey(2000)
    if(k==27):
        break
cv2.destroyAllWindows()

['1493458748-beauty-face-517.jpg', '46950216-large-group-of-people-community-teamwork-concept.jpg', 'adult-braces-brighton-and-hove-1.jpg', 'festival.jpg', 'Find-Child-Care2.jpg', 'pexels-photo-614810.webp']
[[9.9832731e-01 1.6726274e-03 1.1992909e-12 7.6882350e-10]] 0 13-26 99
[[9.9999666e-01 3.2814273e-06 2.1623676e-09 2.1599868e-11]] 0 13-26 99
[[1.24044844e-03 8.39714944e-01 1.58935592e-01 1.08964596e-04]] 1 27-40 83
[[2.1376609e-15 1.3726944e-09 2.8082758e-21 1.0000000e+00]] 3 40+ 100
[[9.9793673e-01 2.0409664e-03 1.8541971e-06 2.0486359e-05]] 0 13-26 99
[[2.7094694e-08 9.9999940e-01 1.8527013e-16 6.3305436e-07]] 1 27-40 99
[[9.7930110e-01 2.0698950e-02 1.0799283e-11 3.4060248e-11]] 0 13-26 97
[[4.8555082e-21 3.3692352e-06 4.0883963e-10 9.9999666e-01]] 3 40+ 99
[[9.9243766e-01 1.2336017e-04 7.4389004e-03 2.0212238e-11]] 0 13-26 99
[[8.4520626e-01 8.1584615e-09 4.0601741e-12 1.5479371e-01]] 0 13-26 84
[[2.3122704e-04 1.2311691e-01 7.0153342e-35 8.7665188e-01]] 3 40+ 87
[[8.4002548e